# Main Concepts

- Gradient Descent: find minimum of cost function (error)
- Steps:
    - parameter initialization
    - find gradients: taking partial derivatives with respect to params or including bias term in feature vector
    - update values of weights and bias
    - repeat until iteration meet or cost function converges
- Main components:
    - cost function: regression (MSE, RMSE, MAE)
    - number of epochs: number of times to pass the entire training dataset
    - learning rate
- Note
    - subjective to feature scale: large scale features dominate the change

[source](https://prasad07143.medium.com/variants-of-gradient-descent-and-their-implementation-in-python-from-scratch-2b3cceb7a1a0)

# Test Data

In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model, model_selection, preprocessing, metrics, datasets

warnings.filterwarnings("ignore")

# Generating random regression data
X, y = datasets.make_regression(n_samples = 1000, n_features = 2, noise = 7, random_state = 16)
print(X.shape, y.shape)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2, random_state = 3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


# Feature Scaling ( Standardization or Z-Score Normalization )
scaler = preprocessing.StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

print(X_train.mean(), X_train.std())
print(X_test.mean(), X_test.std())

(1000, 2) (1000,)
(800, 2) (200, 2) (800,) (200,)
5.551115123125783e-18 1.0
1.7763568394002505e-17 1.0


# Batch Gradient Descent
- Uses the whole batch of training instances to compute gradient
- Con: computationally expensive, might stuck in local minima
- Pro: promise to find optimal solution, low bias and variance

In [ ]:
class BatchGD:

    def __init__(self, max_iter = 1000, lr = 0.001, tol = 1e-6):
        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.weights = None
        self.bias = None
    
    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.zeros(n)
        self.bias = 0
        loss = []

        for i in range(self.max_iter):
            y_pred = np.dot(X, self.weights) + self.bias
            mse = (1/m) * np.sum(np.square(y_pred - y))
            loss.append(mse)

            dw = (1/m) * np.dot(X.T, (y_pred - y))
            db = (1/m) * np.sum(y_pred - y)

            temp = self.weights.copy()
            self.weights -= self.lr*dw
            self.bias -= self.lr*db

            if i % 100 == 0:
                print(f"Iteration {i}, Loss: {mse}")
            
            if all(abs(self.weights - temp) <= self.tol):
                print(f"Convergence reached at iteration {i}")
                break

    def predict(self, X):
        y_pred = np.dot(X, self.weights) + self.bias
        return y_pred

In [18]:
# Training and evaluation ( Batch GD )

lr = BatchGD(max_iter = 10000, lr = 0.01)
lr.fit(X_train, y_train)

print(f"Weights (Coefficients): {lr.weights}\nBias (Intercept): {lr.bias}")

y_test_pred = lr.predict(X_test)
r2_score = metrics.r2_score(y_test, y_test_pred)

print("R-Square (Coefficient of determination): {score}".format(score = r2_score))

Iteration 0, Loss: 4695.039202469317
Iteration 100, Loss: 624.9014751951335
Iteration 200, Loss: 122.3229556633559
Iteration 300, Loss: 60.209367593174086
Iteration 400, Loss: 52.5246998730196
Iteration 500, Loss: 51.572785218556675
Iteration 600, Loss: 51.454699857083725
Iteration 700, Loss: 51.44002671206418
Iteration 800, Loss: 51.4381998832959
Iteration 900, Loss: 51.437971925882906
Iteration 1000, Loss: 51.43794340647068
Iteration 1100, Loss: 51.437939827779914
Iteration 1200, Loss: 51.43793937718558
Convergence reached at iteration 1287
Weights (Coefficients): [54.59449546 38.50982028]
Bias (Intercept): 0.07979662940059772
R-Square (Coefficient of determination): 0.9852928347328443


In [19]:
# sklearn implementation

model = linear_model.LinearRegression()
model.fit(X_train, y_train)

print(f"Weights (Coefficients): {model.coef_}\nBias (Intercept): {model.intercept_}")

r2_score = model.score(X_test, y_test)
print("R-Square (Coefficient of determination): {score}".format(score = r2_score))

Weights (Coefficients): [54.59459269 38.50985045]
Bias (Intercept): 0.07979681999767206
R-Square (Coefficient of determination): 0.9852928016810484


# Stochastic Gradient Descent
- Only use one sample randomly from all training instances at a time
- Pro: computationally efficient, less likely to get stuck in local minima (irregular updates might help bounce off)
- Con: high bias and variance (likely to overfit), irregular updates --> gradually reducing learning rate
- Note: need to shuffle the data first to prevent embeded patterns

In [24]:
class StochasticGD:

    def __init__(self, max_iter = 1000, t0 = 5, t1 = 50, tol = 1e-7):
        self.max_iter = max_iter
        self.t0 = t0 # initial learning rate
        self.t1 = t1 # how quicly the learning rate decreases
        self.tol = tol
        self.weights = None
        self.bias = None
    
    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.random.rand(n)
        self.bias = np.random.rand(1)[0]
        losses, flag = [], False

        for epoch in range(1, self.max_iter + 1):
            # randomly shuffle the data
            random_idx = np.random.permutation(m)
            X, y = X[random_idx], y[random_idx]
            
            if flag:
                break

            if epoch % 10 == 0:
                print(f"Epoch: {epoch}\tLoss: {losses[epoch-1]}")
            
            for i in range(m):
                # randomly select one sample
                rand_idx = np.random.randint(0, m)
                X_i = X[rand_idx: rand_idx + 1]
                y_i = y[rand_idx: rand_idx + 1]

                y_pred = np.dot(X_i, self.weights) + self.bias
                loss = np.sum(np.square(y_pred - y_i))
                losses.append(loss)

                dw = np.dot(X_i.T, y_pred - y_i)
                db = np.sum(y_pred - y_i)
                temp = self.weights.copy()
                lr = self.learning_rate(m * epoch + i)
                self.weights -= lr * dw
                self.bias -= lr * db

                if all(abs(self.weights - temp) <= self.tol):
                    print(f"Convergence reached at epoch {epoch}, iteration {i}")
                    flag = True
                    break
        
    def learning_rate(self, t):
        return self.t0 / (t + self.t1)

    def predict(self, X):
        y_pred = np.dot(X, self.weights) + self.bias
        return y_pred

In [25]:
# Training and evaluation ( Stochastic GD )

lr = StochasticGD(max_iter = 50, tol = 1e-8)
lr.fit(X_train, y_train)

print(f"Weights (Coefficients): {lr.weights}\nBias (Intercept): {lr.bias}")

y_test_pred = lr.predict(X_test)
r2_score = metrics.r2_score(y_test, y_test_pred)

print("R-Square (Coefficient of determination): {score}".format(score = r2_score))


Epoch: 10	Loss: 1970.081006942571
Epoch: 20	Loss: 3388.6035069600257
Epoch: 30	Loss: 1048.025169320888
Epoch: 40	Loss: 19987.961561243857
Epoch: 50	Loss: 10714.48098531692
Weights (Coefficients): [54.64361311 38.51682826]
Bias (Intercept): 0.0502002727646547
R-Square (Coefficient of determination): 0.9852789665256142


In [23]:
# Implementation through Sklean API

model = linear_model.SGDRegressor()
model.fit(X_train, y_train)

print(f"Weights (Coefficients): {model.coef_}\nBias (Intercept): {model.intercept_}")

r2_score = model.score(X_test, y_test)

print("R-Square (Coefficient of determination): {score}".format(score = r2_score))

Weights (Coefficients): [54.54019941 38.47934055]
Bias (Intercept): [0.0721226]
R-Square (Coefficient of determination): 0.9852960135227344


# MiniBatch Gradient Descent
- choose random subset of the entire training set for gradient computation (e.g. 32 samples)
- Pro: provides a performance boost with hardware optimization, with GPU
- Con: may be stuck in local minima
- Note: shuffle taining data to get rid of high bias/variance

In [26]:
class MiniBatchGD:
    def __init__(self, max_iter = 100, lr = 0.001, tol = 1e-6, batch_size = 32):
        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.batch_size = batch_size
        self.weights = None
        self.bias = None
    
    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.random.rand(n)
        self.bias = np.random.rand(1)[0]
        costs, flag = [], False

        for epoch in range(1, self.max_iter + 1):
            perm_idx = np.random.permutation(m)
            X, y = X[perm_idx], y[perm_idx]

            if flag:
                break

            if epoch % 100 == 0:
                print(f"Epoch: {epoch}\tLoss: {costs[epoch-1]}")

            for i in range(0, m, self.batch_size):
                X_mini = X[i: i + self.batch_size]
                y_mini = y[i: i + self.batch_size]

                y_pred = np.dot(X_mini, self.weights) + self.bias
                cost = 1 / self.batch_size * np.sum(np.square(y_pred - y_mini))
                costs.append(cost)

                dw = 1 / self.batch_size * np.dot(X_mini.T, (y_pred - y_mini))
                db = 1 / self.batch_size * np.sum(y_pred - y_mini)
                temp = self.weights.copy()
                self.weights -= self.lr * dw
                self.bias -= self.lr * db

                if all(abs(self.weights - temp) <= self.tol):
                    print(f"Convergence reached at epoch {epoch}, iteration {i}")
                    flag = True
                    break
            

    def predict(self, X):
        y_pred = np.dot(X, self.weights) + self.bias
        return y_pred

In [27]:
# Training and evaluation ( MiniBatch GD )

lr = MiniBatchGD(max_iter = 1000, tol = 1e-7)
lr.fit(X_train, y_train)

print(f"Weights (Coefficients): {lr.weights}\nBias (Intercept): {lr.bias}")

y_test_pred = lr.predict(X_test)
r2_score = metrics.r2_score(y_test, y_test_pred)

print("R-Square (Coefficient of determination): {score}".format(score = r2_score))

Epoch: 100	Loss: 3099.767009122487
Epoch: 200	Loss: 2765.425712432953
Epoch: 300	Loss: 2552.30274655026
Epoch: 400	Loss: 3035.4496211215064
Epoch: 500	Loss: 1566.4481698550503
Epoch: 600	Loss: 1061.3291798904215
Epoch: 700	Loss: 576.9637305208013
Epoch: 800	Loss: 927.9963193487199
Epoch: 900	Loss: 451.38913783901785
Epoch: 1000	Loss: 495.3531838985797
Weights (Coefficients): [54.59459502 38.51002504]
Bias (Intercept): 0.07999148008373759
R-Square (Coefficient of determination): 0.9852929366749039


In [28]:
# Implementation through Sklean API

def form_batches(X, y, batch_size = 32):
    m, n = X.shape
    X_batches, y_batches = [], []
    
    for i in range(0, m, batch_size):
        perm_idx = np.random.permutation(m)
        X, y = X[perm_idx], y[perm_idx]
        
        X_batch = X[i: i + batch_size]
        y_batch = y[i: i + batch_size]

        if X_batch.shape[0] < batch_size:
            continue
        
        X_batches.append(X_batch)
        y_batches.append(y_batch)
        
    return np.array(X_batches), np.array(y_batches)

model = linear_model.SGDRegressor()
X_batches, y_batches = form_batches(X_train, y_train)

for X, y in zip(X_batches, y_batches):
    model.partial_fit(X, y)

print(f"Weights (Coefficients): {model.coef_}\nBias (Intercept): {model.intercept_}")

r2_score = model.score(X_test, y_test)
print("R-Square (Coefficient of determination): {score}".format(score = r2_score))

Weights (Coefficients): [48.63287984 34.75164334]
Bias (Intercept): [0.31491738]
R-Square (Coefficient of determination): 0.9737659442623807
